In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../utilities')
from basic_utilities import *
from model_utilities import *
from pandas.plotting import scatter_matrix
from ml_utilities import *
import scipy.stats as stats
import pdb
import os
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import pickle
sys.path.append('../Data_prep')
from data_config import *
sys.path.append('../../../../infrastructure/tools')
from feature_engineering import feature_engineering
from utilities import utilities
import re
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pdb

In [2]:
train_sample = pd.read_csv('../../../data/processed_data/train_sample_segment0_logit.csv')

In [3]:
train_sample.shape

(23973, 135)

In [4]:
get_knots = read_yaml_file('../Data_prep/1d_knots_segment0.yaml')['get_knots']

In [5]:
knots_vars = []
for var in get_knots:
    matching_strings = [s for s in train_sample.columns.tolist() if re.search(f"{var}_", s)]
    knots_vars += matching_strings

In [6]:
get_dummy = read_yaml_file('../Data_prep/dummy_grouping_segment0_draft.yaml')['get_dummies']

In [7]:
dummy_vars = []
for var in get_dummy:
    matching_strings = [s for s in train_sample.columns.tolist() if re.search(f"{var}_", s)]
    dummy_vars += matching_strings

In [8]:
binning_features = read_yaml_file('../Data_prep/binning_features_segment0.yaml')['binning_features']

In [9]:
binning_features = [f.split(',')[0] for f in binning_features]

In [19]:
bin_vars = []
for var in binning_features:
    matching_strings = [s for s in train_sample.columns.tolist() if re.search(f"{var}_bin:", s)]
    bin_vars += matching_strings

In [23]:
bin_vars

['bank_branch_count_8w_bin:1',
 'bank_branch_count_8w_bin:nan',
 'days_since_request_bin:1',
 'days_since_request_bin:nan',
 'session_length_in_minutes_bin:1',
 'session_length_in_minutes_bin:2',
 'session_length_in_minutes_bin:nan']

In [25]:
spec1 = knots_vars + dummy_vars + bin_vars
spec1 = sorted(list(set(spec1)))

In [27]:
Xtrain = train_sample[spec1].astype('float')
ytrain = train_sample['fraud_bool']
weights = train_sample['sample_weight']

In [31]:
glm = sm.GLM(ytrain, Xtrain, family=sm.families.Binomial(), var_weights=weights)
model1 = glm.fit()
print(model1.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:             fraud_bool   No. Observations:                23973
Model:                            GLM   Df Residuals:                    23928
Model Family:                Binomial   Df Model:                           44
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -14282.
Date:                Wed, 11 Jun 2025   Deviance:                       28563.
Time:                        18:55:00   Pearson chi2:                 1.94e+05
No. Iterations:                   100   Pseudo R-squ. (CS):             0.2694
Covariance Type:            nonrobust                                         
                                              coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------

In [37]:
model2 = model1
while True:
    res = utilities.get_beta_n_pvalues(model2)
    spec = list(res['feature'].values)
    newspec = sorted(res.loc[(pd.notna(res['p-values'])) & (res['p-values']<0.05),'feature'])
    if len(newspec) == len(spec):
        break
    else:
        glm = sm.GLM(ytrain, Xtrain[newspec], family=sm.families.Binomial(), var_weights=weights)
        model2 = glm.fit()

In [39]:
utilities.get_beta_n_pvalues(model2)

,feature,coef,abs_coef,p-values
20,name_email_similarity_2,2.474962,2.474962,1.209563e-07
19,name_email_similarity_1,-2.253001,2.253001,6.576633e-65
21,name_email_similarity_3,-2.191540,2.191540,1.048572e-05
14,housing_status_0,-1.569742,1.569742,2.333244e-49
7,device_distinct_emails_8w_0,1.424414,1.424414,5.136675e-94
18,income_2,1.349201,1.349201,3.112199e-57
8,device_os_0,1.184045,1.184045,2.039565e-191
11,employment_status_3,-1.078558,1.078558,6.233653e-06
16,housing_status_4,-1.077380,1.077380,3.450356e-75
15,housing_status_2,-0.893357,0.893357,6.500122e-67


In [41]:
print(model2.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:             fraud_bool   No. Observations:                23973
Model:                            GLM   Df Residuals:                    23944
Model Family:                Binomial   Df Model:                           28
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -14297.
Date:                Wed, 11 Jun 2025   Deviance:                       28594.
Time:                        18:55:09   Pearson chi2:                 1.95e+05
No. Iterations:                    22   Pseudo R-squ. (CS):             0.2685
Covariance Type:            nonrobust                                         
                                         coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
bank

In [ ]:
#spec3

In [ ]:
#testset